In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from scipy.stats import *

from bokeh.plotting import figure
from bokeh.io import output_notebook, show#, output_file #Descomentar para guardar el archivo .html
from bokeh.models.widgets import Panel, Tabs
output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv("regLin.csv")

In [3]:
def sets(data_frame):
    entrenamiento, prueba = train_test_split(data_frame, test_size=0.2)#, random_state=18)
    return entrenamiento, prueba

In [4]:
entrenamiento, prueba = sets(df)

In [5]:
entrenamiento_y = entrenamiento.y.as_matrix()
entrenamiento_x = entrenamiento.X.as_matrix()
prueba_y = prueba.y.as_matrix()
prueba_x = prueba.X.as_matrix()

In [6]:
unos = np.ones(len(entrenamiento_x))
matriz = np.dstack((unos,entrenamiento_x))
matriz = matriz[0,:]

In [7]:
entrenamiento_y_n = entrenamiento_y/max(entrenamiento_x)
entrenamiento_x_n = entrenamiento_x/max(entrenamiento_y)
prueba_y_n = prueba_y/max(df.X)
prueba_x_n = prueba_x/max(df.y)

In [8]:
unos = np.ones(len(entrenamiento_x))
matriz_n = np.dstack((unos,entrenamiento_x_n))
matriz_n = matriz_n[0]

In [9]:
def LMS(eta):
    """
    Función de eta
    """
    
    w = np.array([1.,1.])

    error = []
    W_0 = []
    W_1 = []

    contador = 0

    while (sum([np.dot(w,matriz_n[i]) - entrenamiento_y_n[i] for i in range(0,len(entrenamiento_y))]))**2 > 0.01:
        error.append((sum([np.dot(w,matriz_n[i]) - entrenamiento_y_n[i] for i in range(0,len(entrenamiento_y))]))**2)
        W_0.append(w[0])
        W_1.append(w[1])
        w[0] = w[0] + (eta * (sum([(entrenamiento_y_n[i] - np.dot(w,matriz_n[i])) * matriz_n[i,0] for i in range(0,len(entrenamiento_y_n))])/len(entrenamiento_y_n)))
        w[1] = w[1] + (eta * (sum([(entrenamiento_y_n[i] - np.dot(w,matriz_n[i])) * matriz_n[i,1] for i in range(0,len(entrenamiento_y_n))])/len(entrenamiento_y_n)))
        contador += 1
        
    return error, w, W_0, W_1, contador

In [10]:
def graficar(eta):
    
    error, w, W_0, W_1, contador = LMS(eta)
    
    s1 = figure(width=750, plot_height=500, title=None)
    s1.circle(entrenamiento_x_n,entrenamiento_y_n, size=5, color="red", alpha=0.5)
    s1.line(np.sort(entrenamiento_x_n), [w[1]*(np.sort(entrenamiento_x_n))[i]+w[0] for i in range(0,len(entrenamiento_x_n))], line_width=1.5, color="navy", alpha=0.7)
    s1.title.text = "Regresión de entrenamiento"
    s1.xaxis.axis_label = "X"
    s1.yaxis.axis_label = "y"
    tab1 = Panel(child=s1, title="Entrenamiento")

    s2 = figure(width=750, plot_height=500, title=None)
    s2.circle(prueba_x_n,prueba_y_n, size=5, color="red", alpha=0.5)
    s2.line(np.sort(prueba_x_n), [w[1]*(np.sort(prueba_x_n))[i]+w[0] for i in range(0,len(prueba_x_n))], line_width=1.5, color="navy", alpha=0.7)
    s2.title.text = "Regresión con la prueba"
    s2.xaxis.axis_label = "X"
    s2.yaxis.axis_label = "y"
    tab2 = Panel(child=s2, title="Prueba")

    s3 = figure(width=750, plot_height=500, title=None)
    s3.circle(W_0,error, size=5, color="red", alpha=0.5)
    s3.title.text = "Errores"
    s3.xaxis.axis_label = "X"
    s3.yaxis.axis_label = "y"
    tab3 = Panel(child=s3, title="Errores w0")

    s4 = figure(width=750, plot_height=500, title=None)
    s4.circle(W_1[20:len(W_1)],error[20:len(W_1)], size=5, color="red", alpha=0.5)
    s4.title.text = "Errores"
    s4.xaxis.axis_label = "X"
    s4.yaxis.axis_label = "y"
    tab4 = Panel(child=s4, title="Errores w1")

    tabs = Tabs(tabs=[ tab1, tab2, tab3, tab4])
    
    show(tabs)
    
    print('Requirió {} iteraciones').format(contador)


In [11]:
graficar(0.05)

Requirió 784 iteraciones


In [12]:
graficar(0.1)

Requirió 367 iteraciones


In [13]:
graficar(1)

Requirió 34 iteraciones


In [14]:
error, w, W_0, W_1, contador = LMS(0.05)

In [15]:
w

array([ 0.02445317,  0.41826217])

In [16]:
w[0] * max(entrenamiento_x)

2.4443289651120672

In [17]:
w[1] * max(entrenamiento_x_n)

0.60737194056901289